# 11-HDBSCAN-Generated Cluster Metric

In this notebook we determine a metric for HDBSCAN-generated clusters. This metric is a surrogate for the missing DBSCAN's ε parameter that determines the maximum reachability distance for points in the cluster. Deriving such a metric is useful when determining H3 hexagon sizes to seamlessly cover the cluster, the inflate size for a concave hull-generated shape, or the maximum radius for bubble shaping.

**Requirements:**

- Please run the `05-clustering-hdbscan.ipynb` notebook first and its dependencies.
- Recommended install: [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html). Enable using `jupyter nbextension enable --py widgetsnbextension --sys-prefix` for Jupyter Notebook and `jupyter labextension install @jupyter-widgets/jupyterlab-manager` for Jupyter Lab.

In [ ]:
import math
import numpy as np
import pandas as pd
import folium
import ipywidgets as widgets
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as stats
import pyproj
import utm
import h3.api.numpy_int as h3

from folium.vector_layers import PolyLine, CircleMarker, Polygon
from db.api import VedDb
from geo.math import vec_haversine, square_haversine, num_haversine
from geo.hulls import ConcaveHull
from shapely.geometry import Polygon as PolygonShape, Point as PointShape
from shapely.ops import unary_union, transform
from fitter import Fitter, get_common_distributions

Select the cluster identifier to analyze below. These identifiers match the ones in the `cluster_id` column of the `cluster` database table.

In [ ]:
cluster_id = 40

Now, we instantiate the database object.

In [ ]:
db = VedDb()

The `get_cluster_locations` function retrieves a list of latitude and longitude pairs for all points in the given cluster. The single parameter is the cluster identifier, as generated by the database.

In [ ]:
locations = db.get_cluster_locations(cluster_id)
locations.shape

Calling the `square_haversine` function on the latitude and longitude lists returns a symmetric square matrix containing the pairwise distances between all locations. The function calculates these distances using a vectorized version of the Haversine distance.

In [ ]:
dist = square_haversine(locations[:, 0], locations[:, 1])

The `non_zero` function is a helper to filter out the diagonal zeros of each column in the distance matrix.

In [ ]:
def non_zero(arr):
    return arr[np.where(arr != 0)]

Now, we use it to extract the list of minimum values on the function below.

In [ ]:
def get_per_location_minimums(locations):
    dist = square_haversine(locations[:, 0], locations[:, 1])
    minimums = [non_zero(dist[:, i]).min() for i in range(dist.shape[1])]
    return minimums   

One could argue that the HDBSCAN equivalent of DBSCAN's $\epsilon$ is the maximum of said minimum distances. For this case that value is:

In [ ]:
max(get_per_location_minimums(locations))

As we will see below, and for the purposes of finding a good measure for the cluster buffers, this might actually be an overestimation.

## Statistics-Based Approach

We now turn our exploration to using the minimum distance statistics to derive a reasonable cluster metric. The first approach is to assume that the minimum distances have a Normal distribution which, as we will later see, is _not correct_.

The function below calculates the cluster distance metric, _assuming_ a Normal distribution, as $\mu+2\sigma$, that would correspond to roughly 95% of distance distribution.

In [ ]:
def get_cluster_matrix_sig(locations, sigma_factor = 2.0):
    minimums = get_per_location_minimums(locations)
    return np.average(minimums) + np.std(minimums) * sigma_factor

In [ ]:
cluster_matrix_sig = get_cluster_matrix_sig(locations)
cluster_matrix_sig

By researching into the typical minimum distance distribution (see Notebook 12), we find that, for this dataset, most cluster minimum distances follow either a Log-Normal or a Gamma distribution. We can now see how the same approach as the Normal behaves under a Log-Normal.

The function below assumes that the distance distrubution follows a Log-Normal distribution, computing the metric as $m+2s$, where $m=e^{\mu+\frac{\sigma^{2}}{2}}$ and $s=m\sqrt{e^{\sigma^{2}}}$

In [ ]:
def get_cluster_matrix_logsig(locations, sigma_factor = 2.0):
    minimums = get_per_location_minimums(locations)
    logs = np.log(minimums)
    miu = np.average(logs)
    sig = np.std(logs)
    m = np.exp(miu + sig * sig / 2)
    s = np.sqrt((np.exp(sig * sig) - 1)) * m
    return m + s * sigma_factor

In [ ]:
cluster_metric = get_cluster_matrix_logsig(locations, 2)
cluster_metric

In [ ]:
def get_cluster_metric(minimums, factor=2.0):
    logs = np.log(minimums)
    mu = np.average(logs)
    sigma = np.std(logs)
    m = math.exp(mu + sigma * sigma / 2)
    s = math.sqrt((np.exp(sigma * sigma) - 1)) * m
    return m + s * factor

In [ ]:
get_cluster_metric(get_per_location_minimums(locations))

The cell below times the execution of the per-location minimum calculation.

In [ ]:
%%timeit
get_cluster_metric(get_per_location_minimums(locations))

Here is the map representation for the cluster's locations.

In [ ]:
def fit_bounding_box(html_map, bb_list):
    if isinstance(bb_list, list):
        ll = np.array(bb_list)
    else:
        ll = bb_list
        
    min_lat, max_lat = ll[:, 0].min(), ll[:, 0].max()
    min_lon, max_lon = ll[:, 1].min(), ll[:, 1].max()
    html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    return html_map

In [ ]:
def draw_locations(html_map, locations):
    for l in locations:
        c = CircleMarker(l.tolist(), radius=2, color="red", fill="red", opacity=0.5)
        c.add_to(html_map)
    return html_map

In [ ]:
html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20)

html_map = draw_locations(html_map, locations)
    
fit_bounding_box(html_map, locations)

## Network Theory Approach
We can alternatively look at this problem through the perspective of network theory. We start by generating a fully-connected undirected graph where each node is a location, and we weight the edges using the calculated distances.

In [ ]:
g = nx.Graph()
n = locations.shape[0]
g.add_nodes_from(range(n))
g.add_edges_from([(i, j, {'weight': dist[i, j]}) for i in range(n-1) for j in range(i + 1, n)])

We now calculate the minimum spanning tree of the graph, using Prim's algorithm. The resulting edge weights correspond to the list of minimum distances we calculated above.

In [ ]:
mst = nx.minimum_spanning_tree(g, algorithm="prim", weight="weight")

Here is how the minimum spanning tree looks like.

In [ ]:
nx.draw(mst, node_size=20, alpha=0.6)

The code below retrieves the minimum spanning tree edge distances. Note how we must use the distance matrix calculated above.

In [ ]:
min_dist = [dist[e[0], e[1]] for e in mst.edges()]

max(min_dist)

Again, we package all of this code into a single function:

In [ ]:
def get_graph_mst_minimums(locations):
    n = locations.shape[0]
    g = nx.Graph()
    
    dist = square_haversine(locations[:, 0], locations[:, 1])

    g.add_nodes_from(range(locations.shape[0]))
    g.add_edges_from([(i, j, {'weight': dist[i, j]}) for i in range(n) for j in range(i + 1, n)])
            
    mst = nx.minimum_spanning_tree(g, algorithm='prim', weight="weight")
    min_dist = [dist[e[0], e[1]] for e in mst.edges() if dist[e[0], e[1]] > 0.0]
    return min_dist

In [ ]:
cluster_metric = get_cluster_metric(get_graph_mst_minimums(locations))
cluster_metric

As you can see, the result is a bit larger than the per-location one. You can iterate through other clusters by re-running the notebook after changing the cluster identifier above. Unfortunately, this approach is a full order of magnitude slower that the distance matrix approach.

In [ ]:
%%timeit
get_cluster_metric(get_graph_mst_minimums(locations))

The function below, `map_with_mst`, displays the cluster along with an overlayed representation of the minimum spanning tree.

In [ ]:
def map_with_mst(locations, mst):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    for e in mst.edges():
        line = [[locations[e[0], 0], locations[e[0], 1]], [locations[e[1], 0], locations[e[1], 1]]]
        l = PolyLine(line, weight=2, opacity=0.5)
        l.add_to(html_map)
        
    html_map = draw_locations(html_map, locations)

    return fit_bounding_box(html_map, locations)

In [ ]:
map_with_mst(locations, mst)

## Minimum Distance Distribution

Now we inspect how the minimum distance distribution looks like, and what distribution best fits. Here, we use the `fitter` package to discover the best-fitting distributions.

In [ ]:
f = Fitter(min_dist, distributions=get_common_distributions())
f.fit()
f.summary()

In [ ]:
f.get_best()

## Log Normal Distribution Fit

In [ ]:
fig, ax = plt.subplots()
c, b, p = ax.hist(min_dist, bins=20)

fit_alpha, fit_loc, fit_beta = stats.lognorm.fit(min_dist)

x = np.linspace(0.0, max(b))
y = stats.lognorm.pdf(x, fit_alpha, fit_loc, fit_beta)

ax2 = ax.twinx()
ax2.plot(x, y, alpha=0.6, color="red")

ax.set(xlabel='Distance (m)', ylabel='Count',
       title='Minimum Distances - Log Normal')
plt.show()

### Gamma Distribution Fit
Here we try to fit the minimum distances to the Gamma distribution

In [ ]:
fig, ax = plt.subplots()
c, b, p = ax.hist(min_dist, bins=20)

fit_alpha, fit_loc, fit_beta = stats.gamma.fit(min_dist)

x = np.linspace(0.0, max(b))
y = stats.gamma.pdf(x, fit_alpha, fit_loc, fit_beta)

ax2 = ax.twinx()
ax2.plot(x, y, alpha=0.6, color="red")

ax.set(xlabel='Distance (m)', ylabel='Count',
       title='Minimum Distances - Gamma')
plt.show()

# Concave Hull

In [ ]:
def get_concave_hull_shape(locations, k):
    hull = ConcaveHull([[x[1], x[0]] for x in locations])
    shape = hull.calculate(k)
    shape_latlng = np.array([[x[1], x[0]] for x in shape])
    return shape_latlng

In [ ]:
def draw_concave_hull(html_map, locations, k):
    shape_latlng = get_concave_hull_shape(locations, k)
    
    polygon = Polygon(shape_latlng, weight=1, opacity=0.5)
    polygon.add_to(html_map)
    return html_map

In [ ]:
def map_hull(locations, k=3):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    html_map = draw_concave_hull(html_map, locations, k)
    html_map = draw_locations(html_map, locations)

    return fit_bounding_box(html_map, locations)

In [ ]:
map_hull(locations)

In [ ]:
def buffer_in_meters(shape, meters):
    xs, ys, zn, zl = utm.from_latlon(shape[:,1], shape[:,0])

    polygon = PolygonShape(np.array([xs, ys]).T)

    buffer_meters = polygon.buffer(meters)

    xs = np.array([l[0] for l in buffer_meters.exterior.coords])
    ys = np.array([l[1] for l in buffer_meters.exterior.coords])
    
    lats, lngs = utm.to_latlon(xs, ys, zn, zl)

    return lats, lngs

In [ ]:
def map_buffered_hull(locations, cluster_metric, k=3, metric_factor=1.0):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    shape_latlng = get_concave_hull_shape(locations, k)
    polygon = Polygon(shape_latlng, weight=1, opacity=0.5)
    polygon.add_to(html_map)
    
    lats, lngs = buffer_in_meters(np.array([[x[1], x[0]] for x in shape_latlng]), 
                                  cluster_metric * metric_factor)
    buffer_polygon = Polygon(np.array([lats, lngs]).T,
                             weight=2, opacity=0.6, color="blue")
    buffer_polygon.add_to(html_map)

    html_map = draw_locations(html_map, locations)
    
    print(lats.shape[0])
    
    return fit_bounding_box(html_map, shape_latlng)

In [ ]:
map_buffered_hull(locations, cluster_metric)

# Bubbles

In [ ]:
def get_bubbles_shape(locations, metric_factor):
    xs, ys, zn, zl = utm.from_latlon(locations[:,0], locations[:,1])

    points = [PointShape([l[0], l[1]]) for l in zip(xs, ys)]
    bubbles = [point.buffer(cluster_metric * metric_factor) for point in points]
    final_shape = unary_union(bubbles)

    xs = np.array([l[0] for l in final_shape.exterior.coords])
    ys = np.array([l[1] for l in final_shape.exterior.coords])

    lats, lngs = utm.to_latlon(xs, ys, zn, zl)
    shape_latlng = np.array([lats, lngs]).T
    return shape_latlng

In [ ]:
def draw_bubbles(html_map, shape_latlng, metric_factor):
    polygon = Polygon(shape_latlng, weight=2, opacity=0.6)
    polygon.add_to(html_map)
    return html_map

In [ ]:
def map_bubbles(locations, cluster_metric, metric_factor=1.0):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)

    shape_latlng = get_bubbles_shape(locations, metric_factor)

    html_map = draw_bubbles(html_map, shape_latlng, metric_factor)
    html_map = draw_locations(html_map, locations)
    
    print(shape_latlng.shape[0])
    
    return fit_bounding_box(html_map, shape_latlng)

In [ ]:
map_bubbles(locations, cluster_metric, metric_factor=2.0)

# H3

In [ ]:
def get_h3_resolution(meters):
    h3_side_km = np.array(
        [1107.712591000,
           418.676005500,
           158.244655800,
            59.810857940,
            22.606379400,
             8.544408276,
             3.229482772,
             1.220629759,
             0.461354684,
             0.174375668,
             0.065907807,
             0.024910561,
             0.009415526,
             0.003559893,
             0.001348575,
             0.000509713])
    kms = meters / 1000.0
    return np.argwhere(h3_side_km < kms)[0, 0]

In [ ]:
h3_res = get_h3_resolution(cluster_metric)

In [ ]:
cluster_metric, h3_res

In [ ]:
def get_hexagon(h):
    geo_lst = list(h3.h3_to_geo_boundary(h))
    geo_lst.append(geo_lst[0])
    return np.array(geo_lst)

In [ ]:
def create_map_polygon(xy, tooltip='',
                       color='#3388ff',
                       opacity=0.7,
                       # fill_color='#3388ff',
                       # fill_opacity=0.4, 
                       weight=3):
    points = [[x[0], x[1]] for x in xy]
    polygon = folium.vector_layers.Polygon(locations=points,
                                           tooltip=tooltip,
                                           # fill=True,
                                           color=color,
                                           # fill_color=fill_color,
                                           # fill_opacity=fill_opacity,
                                           weight=weight,
                                           opacity=opacity)
    return polygon

In [ ]:
def get_merged_hexagons(locations, h3_res):
    bb_list = []  # List for the bounding-box calculation
    polygons = []
    hexes = list(set([h3.geo_to_h3(l[0], l[1], h3_res) for l in locations]))
    
    for h in hexes:
        points = get_hexagon(h)
        xy = [[x[1], x[0]] for x in points]
        xy.append([points[0][1], points[0][0]])
        polygons.append(PolygonShape(xy))
        bb_list.extend(points)
        
    merged = unary_union(polygons)
    return merged, bb_list

In [ ]:
def create_map_cluster(locations, cluster_metric, metric_factor=1.0):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    h3_res = get_h3_resolution(cluster_metric * metric_factor)

    merged, bb_list = get_merged_hexagons(locations, h3_res)
    
    if merged.geom_type == "MultiPolygon":
        max_len = 0
        largest = None
        for geom in merged.geoms:
            xy = geom.exterior.coords.xy
            lxy = list(zip(xy[1], xy[0]))
            create_map_polygon(lxy).add_to(html_map)
    elif merged.geom_type == "Polygon":
        xy = merged.exterior.coords.xy
        lxy = list(zip(xy[1], xy[0]))
        create_map_polygon(lxy).add_to(html_map)

    html_map = draw_locations(html_map, locations)

    return fit_bounding_box(html_map, bb_list)

In [ ]:
create_map_cluster(locations, cluster_metric, metric_factor=3.0)

In [ ]:
def create_full_map(locations):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    html_map = draw_locations(html_map, locations)
    return html_map